# User for Genre

- User for genre toma como parámetro un género particular, y retorna el usuario  con mayor tiempo jugado para dicho genero.

- Para evitar consultas lentas, crearemos una tabla auxiliar que sera agrupada por año y jugador, donde agruparemos los generos en un str y sumaremos el tiempo jugado del usuario.

- **Observación:** No tenemos la información sobre el año en que un usuario jugó un juego, por lo que vamos a tomar el año de lanzamiento del juego como el año en que el usuario jugó el juego. 

In [13]:
import pandas as pd
import os

path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Carga de data

In [14]:
path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Cargamos la data necesaria para solucionar este problema.
steam_colums = ['id','genres','release_year']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)

user_item_cols = ['item_id','user_id','playtime_forever']
user_items = pd.read_csv(path_user, usecols = user_item_cols)


In [17]:
steam_games.head(3)

,genres,steam_id,release_year
0,"Strategy, Action, Indie, Casual, Simulation",761140.0,2018
1,"Free to Play, Strategy, Indie, RPG, Card Game,...",643980.0,2018
2,"Free to Play, Simulation, Sports, Casual, Indi...",670290.0,2017


In [16]:
user_items.head(3)

,user_id,playtime_forever,item_id
0,76561197970982479,0.0,10.0
1,76561197970982479,0.0,20.0
2,76561197970982479,0.0,30.0


## Creación de tabla consulta

- En esta sección combinaremos steam_games  con user_items, por medio de sus ids. Luego agruparemos por año de lanzamiento y grupo de genero comun, para calcular la suma de tiempos jugados por dicho genero.

In [6]:

steam_users = steam_games.merge(user_items,
                              left_on='steam_id', 
                              right_on= 'item_id',
                              how='inner').drop(columns=['steam_id','item_id'])

In [7]:
steam_users.sample(5)

,genres,release_year,user_id,playtime_forever
3460013,"Sandbox, Adventure, Survival, 2D, Multiplayer,...",2011,76561198067655618,3.6
544180,"Tower Defense, Strategy, FPS, Action, Co-op, I...",2011,OMGitsAliAA,0.0
239773,"World War II, FPS, Action, Shooter, Simulation...",2009,spinefarm,0.0
2932774,"RPG, Fantasy, Classic, Isometric, Party-Based ...",2013,76561197971327741,0.0
195742,"Horror, Action, Sci-fi, Space, Third Person, S...",2008,sprocketw,0.0


- Por eficiencia en la consulta vamos a agrupar por año de lanzamiento y usuario.
Notamos que quedan dos columans sin agrupar, genres (str) y playtime_forever. Al realizar la agrupación por suma, vamos a acumular en playtime_forever las horas jugadas por un usuario en un año determinado; 

- Por otra parte genres concatenará cada string que contiene las columnas agrupadas. Esta técnica nos permitira seguir teniendo toda la información necesaria. Posteriormente vamos a transformar la columna genres para eliminar los generos repetidos en cada registro.

In [8]:
tabla_endpoint_2 = steam_users.groupby(['release_year','user_id']).sum(numeric_only=False).reset_index()

In [9]:
tabla_endpoint_2.head(10)

,release_year,user_id,genres,playtime_forever
0,1983,2Ta4,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
1,1983,76561197960324641,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
2,1983,76561197966936422,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
3,1983,76561197968887720,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
4,1983,76561197969020980,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
5,1983,76561197971401137,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
6,1983,76561197971951483,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
7,1983,76561197972452208,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
8,1983,76561197973470219,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
9,1983,76561197975369524,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0


- Así observamos que la tabla resultante tendra un tamaño considerablemente menor y cada usuario que se encuentre en estas lista, tendra los generos jugados por este usuario.

- Ahora vamos a eliminar los generos repetidos en cada registro, esto con el fin de reducir el tamaño del string de cada registro de genres. dado que esta consulta consume bastante memoria, utilicé una tecnica de carga con generadores.

In [48]:
## tomar los generos diferentes de cada uno de los caracteres, esto con el fin de reducir el tamaño del archivo.

def genres_unique(x):
  return ', '.join(set(map(str.strip, x.split(', '))))

tabla_endpoint_2['genres'] = list(genres_unique(fila) for fila in tabla_endpoint_2['genres'])

In [49]:
tabla_endpoint_2.head(10)

,release_year,user_id,genres,playtime_forever
0,1983,2Ta4,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
1,1983,76561197960324641,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
2,1983,76561197966936422,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
3,1983,76561197968887720,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
4,1983,76561197969020980,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
5,1983,76561197971401137,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
6,1983,76561197971951483,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
7,1983,76561197972452208,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
8,1983,76561197973470219,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0
9,1983,76561197975369524,"Quick-Time Events, Classic, Adventure, Retro, ...",0.0


In [55]:
# Me quedo con usuarios que jugaron almenos una hora en steam
# Una persona que jugo menos de una hora en un año completo, no lo considero gammer
tabla_endpoint_2 = tabla_endpoint_2[tabla_endpoint_2['playtime_forever']>=1]

In [12]:
tabla_endpoint_2[tabla_endpoint_2['user_id'] == "REBAS_AS_F-T"]

,release_year,user_id,genres,playtime_forever
17153,1996,REBAS_AS_F-T,"Action, Free to Play, Shoot 'Em Up",0.0
36546,1998,REBAS_AS_F-T,"FPS, Classic, Action, Sci-fi, Singleplayer, Sh...",0.0
50057,1999,REBAS_AS_F-T,"Free to Play, Multiplayer, Co-op, Action, FPS,...",0.0
64835,2000,REBAS_AS_F-T,"Simulation, Strategy, Space, Sci-fi, Space Sim...",0.0
79718,2001,REBAS_AS_F-T,"Free to Play, Action, Indie, Multiplayer, Alie...",0.0
93224,2002,REBAS_AS_F-T,"RPG, First-Person, Classic, Atmospheric, Fanta...",0.0
111602,2003,REBAS_AS_F-T,"Racing, Open World, Great Soundtrack, Arcade, ...",0.6
142419,2004,REBAS_AS_F-T,"FPS, Action, Classic, Shooter, Open World, Ste...",2.4
168447,2005,REBAS_AS_F-T,"Open World, Action, Multiplayer, Crime, Third ...",1.2
211548,2006,REBAS_AS_F-T,"Sandbox, Multiplayer, Funny, Moddable, Buildin...",0.6


## Exportar tabla consulta 2

In [58]:
path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data.csv.gz')

tabla_endpoint_2.to_csv(path_endpoint_2, index = False,compression='gzip')

In [59]:
tabla_endpoint_2

,release_year,user_id,genres,playtime_forever
123,1987,76561197967174031,"Classic, Adventure, Retro, Pixel Graphics, Det...",1.2
1190,1988,comquack,"Classic, Great Soundtrack, RPG, Adventure, Ret...",2.4
1579,1989,76561198014848587,"Classic, Adventure, Retro, Simulation, Strategy",1.2
3705,1993,76561197970628046,"Isometric, Turn-Based, Turn-Based Strategy, Si...",1.2
3729,1993,76561197972443914,"Isometric, Turn-Based, Turn-Based Strategy, Si...",1.2
...,...,...,...,...
763383,2017,zumpo,"Sandbox, First-Person, Third-Person Shooter, M...",8.4
763387,2017,zvanik,"Sandbox, First-Person, Post-apocalyptic, Third...",2.4
763389,2017,zxcvbnm6995,"Sandbox, First-Person, Post-apocalyptic, MMORP...",3.0
763399,2017,zzbrunozz,"Sandbox, First-Person, Post-apocalyptic, Third...",2.4


## Lectura de tabla consulta

**Retornaremos el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año**

In [60]:
path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data.csv.gz')
table_2 = pd.read_csv(path_endpoint_2,compression='gzip')

In [61]:
print(table_2.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214003 entries, 0 to 214002
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   release_year      214003 non-null  int64  
 1   user_id           214003 non-null  object 
 2   genres            214003 non-null  object 
 3   playtime_forever  214003 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.6 MB
None


## Construcción de la consulta
- A continuación vamos a realizar las consultas necesarias para encontrar el usuario con mas horas jugadas por genero.
Para responder este problema vamos a ejemplificar la consulta inspecciónar con un genero particular, en este caso **Action**, luego vamos a construir las consultas para este genero particular

In [62]:
genre = 'Action'

In [63]:
# Filtro la tabla para quedarme únicamente con registros que contengan el género deseado
genre_filter = table_2[table_2['genres'].str.contains(genre)]


In [64]:
genre_filter.head(4)

,release_year,user_id,genres,playtime_forever
1,1988,comquack,"Classic, Great Soundtrack, RPG, Adventure, Ret...",2.4
3,1993,76561197970628046,"Isometric, Turn-Based, Turn-Based Strategy, Si...",1.2
4,1993,76561197972443914,"Isometric, Turn-Based, Turn-Based Strategy, Si...",1.2
5,1993,76561197974745554,"Isometric, Turn-Based, Turn-Based Strategy, Si...",2.4


In [65]:
## Sumamos playtime_forever de cada usuario
df_sum_play_time = genre_filter.groupby('user_id')['playtime_forever'].sum()

In [66]:
pd.DataFrame(df_sum_play_time)

,playtime_forever
user_id,
--000--,37.8
--ace--,16.8
--ionex--,9.6
-2SV-vuLB-Kg,9.6
-404PageNotFound-,50.4
...,...
zzjang2,18.0
zzoptimuszz,19.2
zzydrax,1.2


In [67]:
# Usuario con más horas jugadas en total
user_max_time = df_sum_play_time.idxmax()

print(f'El usurio con mas horas jugadas \npara el genero {genre} es {user_max_time}')


El usurio con mas horas jugadas 
para el genero Action es REBAS_AS_F-T


In [68]:
# Filtrar solo por los registros del usuario con mayor tiempo de juego y me quedo con las columnas release_year y playtime_forever
mask = genre_filter['user_id'] == user_max_time

user_filter = genre_filter[mask][['release_year', 'playtime_forever']]

In [57]:
user_filter.head(5)

,release_year,playtime_forever
17153,1996,0.00
36546,1998,0.02
50057,1999,3.75
64835,2000,0.00
79718,2001,0.18


In [69]:
# Este es el tiempo acumulado jugado por el usuario con mas tiempo jugado con el genero action.
pd.DataFrame(user_filter)

,release_year,playtime_forever
5765,2004,2.4
8479,2005,1.2
30592,2007,2.4
33849,2008,4.2
45240,2009,15.6
61416,2010,19.8
82078,2011,49.8
115943,2012,69.6
147440,2013,109.8
166188,2014,127.2


In [70]:
# Retornaremos como una lista de diccionarios del estilo
# {year : playtime_forever_cum }

target = user_filter.rename(columns={'release_year': 'year', 'playtime_forever': 'total_horas'}).to_dict(orient='records')

In [71]:
target[:5]

[{'year': 2004, 'total_horas': 2.4},
 {'year': 2005, 'total_horas': 1.2},
 {'year': 2007, 'total_horas': 2.4},
 {'year': 2008, 'total_horas': 4.2},
 {'year': 2009, 'total_horas': 15.6}]

In [72]:
print(f"Usuario con más horas jugadas para Género {genre}: es {user_max_time}, \nHoras jugadas:")
target

Usuario con más horas jugadas para Género Action: es REBAS_AS_F-T, 
Horas jugadas:


[{'year': 2004, 'total_horas': 2.4},
 {'year': 2005, 'total_horas': 1.2},
 {'year': 2007, 'total_horas': 2.4},
 {'year': 2008, 'total_horas': 4.2},
 {'year': 2009, 'total_horas': 15.6},
 {'year': 2010, 'total_horas': 19.8},
 {'year': 2011, 'total_horas': 49.8},
 {'year': 2012, 'total_horas': 69.6},
 {'year': 2013, 'total_horas': 109.8},
 {'year': 2014, 'total_horas': 127.2},
 {'year': 2015, 'total_horas': 296.4},
 {'year': 2016, 'total_horas': 366.6},
 {'year': 2017, 'total_horas': 10.2}]

## Función UserForGenre

In [73]:

def user_for_genre(genre :str):
  path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data.csv.gz')
  table_2 = pd.read_csv(path_endpoint_2)

  ## Filtro la tabla para quedarme unicamente con registros que contengan el genero deseado
  genre_filter = table_2[table_2['genres'].str.contains(genre)]
  
  ## usuario con mayor tiempo de juego acumulado
  df_sum_play_time =  genre_filter[['user_id','playtime_forever']].groupby(['user_id']).sum()
  
  # usuario con mas horas jugadas en total
  if len(df_sum_play_time) > 0:
    user_max_time = df_sum_play_time.idxmax().iloc[0]
  else:
    return ('No existe usuario que haya jugado este juego')
  
  ## Filtraremos solo por los registros del usuario con mayor tiempo de juego
  ## nos quedamos con el año de lanzamiento y el tiempo de juego
  mask = genre_filter['user_id'] == user_max_time
  user_filter = genre_filter[mask][['release_year','playtime_forever']]
  
  # Sumamos las horas jugadas del usuario por añoprofile
  user_filter_sum = user_filter.groupby('release_year').sum().reset_index()
  
  target = user_filter_sum.rename(columns={'release_year':'year'})[['year', 'playtime_forever']].to_dict(orient='records')
  return {f"Usuario con más horas jugadas para Género {genre}" : user_max_time, "Horas jugadas": target}

In [74]:
user_for_genre('Action')


{'Usuario con más horas jugadas para Género Action': 'REBAS_AS_F-T',
 'Horas jugadas': [{'year': 2004, 'playtime_forever': 2.4},
  {'year': 2005, 'playtime_forever': 1.2},
  {'year': 2007, 'playtime_forever': 2.4},
  {'year': 2008, 'playtime_forever': 4.2},
  {'year': 2009, 'playtime_forever': 15.6},
  {'year': 2010, 'playtime_forever': 19.8},
  {'year': 2011, 'playtime_forever': 49.8},
  {'year': 2012, 'playtime_forever': 69.6},
  {'year': 2013, 'playtime_forever': 109.8},
  {'year': 2014, 'playtime_forever': 127.2},
  {'year': 2015, 'playtime_forever': 296.4},
  {'year': 2016, 'playtime_forever': 366.6},
  {'year': 2017, 'playtime_forever': 10.2}]}

In [ ]:
user_items['playtime_forever'][user_items['user_id']=='REBAS_AS_F-T'].sum()